<a href="https://colab.research.google.com/github/nitishast/TF/blob/master/NLP_Text_Generation_using_Keras_and_TF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [0]:
import numpy as np
import pandas as pd
import matplotlib as mp
import tensorflow as tf

In [0]:
tf.__version__

In [0]:
%tensorflow_version 2.x

In [0]:
path_to_file = "shakespeare.txt"

In [0]:
text = open(path_to_file,'r').read()

In [0]:
text[2999:30100]

In [0]:
sorted(set(text)) # Return all the unique characters in the text file in a sorted manner. 

In [0]:
vocab = sorted(set(text)) #contains all the characters in the text  

In [0]:
len(vocab) # count the number of characters in the text file. This will be used at the output layer of the RNN

In [0]:
for pair in enumerate(vocab):
  print (pair)

In [0]:
char_to_ind = {char:ind for ind,char in enumerate(vocab)}

In [0]:
ind_to_char = np.array(vocab)

In [0]:
ind_to_char[80]

In [0]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [0]:
encoded_text[:3]

In [0]:
text[2:200]

In [0]:
encoded_text[2:200]

### Defining the sequence length from the lines. To find the structure of the sentence.

In [0]:
print (text[:300])


In [0]:
line = "From fairest creatures we desire increase" #Taking a single line 
lines = '''
From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease
'''

In [0]:
print (len(line))
print (len (lines))
# Here a structured line from the poem or suanet here is of 133 line hence taking it to be of 120 line. 
#This can change for different type of texts. A story or an email can have different sequence legnth

In [0]:
seq_len = 120

In [0]:
total_num_sep = print (len(text)// seq_len + 1)  # Finding the total sequences in the text. Also using + 1 as 0 is the indexing. 

In [0]:
char_dateset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [0]:
type(char_dateset)

In [0]:
print (char_dateset)

In [0]:
for item in char_dateset.take(500):
  print(ind_to_char[item.numpy()])

In [0]:
sequences = char_dateset.batch(batch_size=seq_len+1,drop_remainder=True)
#creating a sequence batches of length 121 from the dataset. Dropping the remaining characters from the text

In [0]:
#Now, creating a target sequences from sequence above by shifting each batch by +1 and storing in tuple object.

In [0]:
def create_seq_targets(seq):
  input_txt = seq[:-1] # "Hello my nam..."
  output_txt = seq[1:] # "ello my name...."
  return input_txt,output_txt

In [0]:
dataset = sequences.map(create_seq_targets)

In [0]:
for input_txt, target_txt in dataset.take(1):
  print(input_txt.numpy())
  print ("".join(ind_to_char[input_txt.numpy()])) ### This is the input sequence 
  print ('\n')
  print (target_txt.numpy())
  print ("".join(ind_to_char[target_txt.numpy()])) # This is the target sequence 
  print ('\n')

### Next target is to create or generate the Training sequence. 
Batches and shuffle. 

In [0]:
# 
batch_size = 128

# Need to define the buffer size for also for randomamising. Else all the text in the memory can be randsomised.
buffer_size = 10000
dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)


In [0]:
type(dataset)

In [0]:
vocab_size = len(vocab)
rnn_neurons=1026
embeded_dim=64

In [0]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [0]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true,y_pred,from_logits=True)


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GRU,Dense

In [0]:
def create_model(vocab_size,embeded_dim,rnn_neurons,batch_size):
  
  model = Sequential()
  model.add(Embedding(vocab_size,embeded_dim,batch_input_shape=[batch_size,None]))
  model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,
              recurrent_initializer='glorot_uniform'))

  model.add(Dense(vocab_size))

  model.compile('adam',loss=sparse_cat_loss)

  return model

In [0]:
model = create_model(vocab_size=vocab_size,embeded_dim=embeded_dim,rnn_neurons=rnn_neurons,
                     batch_size=batch_size)

In [0]:
model.summary()

# Train the model

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  #input_example_batch is the first sequence of letters
  #target_example_batch is ths initial sequence shifted just one  
  example_batch_predections = model(input_example_batch)
  #print (tf.shape(input_example_batch))

In [0]:
example_batch_predections.shape

In [0]:
example_batch_predections[0]

## Grabbing the indices from the tensor and finding the sequence of number generated by the predictor.

In [0]:

sample_indices = tf.random.categorical(example_batch_predections[0],num_samples=1)


## Next with the shape of this tensor, we need to convert it to an array to check out the char by passing in ind_to_char array.

In [0]:
sample_indices = tf.squeeze(sample_indices,axis =-1).numpy()
## reshaping into a numpy array by using squeeze method

In [0]:
sample_indices

In [0]:
ind_to_char[sample_indices]

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True)

In [0]:
## NEw code

import os 
checkpoint_path = "training_1\cp.ckpt" 

checkpoint_dir = os.path.dirname(checkpoint_path)

 # Create checkpoint  callback 
checkpoint_callback =tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_weights_only=True,verbose=1)

In [0]:
epochs = 1

In [0]:
model.fit(dataset,epochs=epochs)

In [0]:
#model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)
model = create_model(vocab_size=vocab_size,embeded_dim=embeded_dim,rnn_neurons=rnn_neurons,
                     batch_size=batch_size)

In [0]:
model.load_weights(model)



In [0]:
model.summary()

In [0]:
def generate_text(model,start_seed,gen_size=500,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters.
  '''

  # defining the number of elements we want to predict
  num_generate = gen_size
  # converting the charater in starting seed string into characters.
  input_eval = [char_to_ind[s] for s in start_seed]
  
  #changing the dimentsion so that its in right shape for our model.'
  input_eval = tf.expand_dims(input_eval,0)

  #empty list to append the text generated by the model
  text_generated = []

  #Need to set the temperature for the randomness in our model.
  #The temeprerature divdes the logits which will be used for the softmax function before passing onto the next
  temperature = temp

  #Resetting the state of model before calling calling the predection clears the hidden layer states.
  model.reset_states()

  #Looping in through each of the characters
  for i in range(num_generate): 

    prediction = model(input_eval)

    #squezzing to undo the expanding dimensions domne above.
    prediction = tf.squeeze(prediction,0)

    #dividing the logits
    prediction = prediction/temperature

    #finding the single digit from the tensor to find the index and pass ot character array to find the real letters.
    #use function random.categorical

    predicted_id = tf.random.categorical(prediction,num_samples=1)[-1:0].numpy()

    #Expand the predected text 
    input_eval = tf.expand_dims(predicted_id,0)

    #append into the list
    text_generated.append(ind_to_char[predicted_id])

  return (start_seed + "".join(text_generated))

In [0]:
print(generate_text(model,"BRUTU",gen_size=500))

In [0]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''

  # Number of characters to generate
  num_generate = gen_size

  # Vecotrizing starting seed text
  input_eval = [char_to_ind[s] for s in start_seed]

  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty list to hold resulting generated text
  text_generated = []

  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected
 
  temperature = temp

  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):

      # Generate Predictions
      print (tf.shape(input_eval))
      predictions = model(input_eval)

      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

In [0]:
print(generate_text(model,"BRUTU",gen_size=500))